Realizaremos el primer proceso de limpieza, teniendo en cuenta los siguientes puntos:

A.- Valores nulos

B.- Valores duplicados

C.- datos faltantes

D.- datos no necesarios

# Análisis exploratorio

El análisis exploratorio nos dará información estructural de los dataframes (tamaño, orden, etc) para poder realizar el primer proceso de limpieza, en donde queden unicamente datos relevantes, para que posteriormente se haga una transformación (reducción).

In [1]:
import pandas as pd
import re
import numpy as np
#Importamos 5 cotizaciones para realizar la parte de limpieza.

cot1 = pd.read_csv('C:/Users/52473/OneDrive/Documentos/SITEC/Cotizaciones (caratula)/Capa1Extract/2023/Enero/CTQ-787 ELECTROSERVICIOS (DESCARGA Y ARRASTRE DE MAQUINA).csv')
cot2 = pd.read_csv('C:/Users/52473/OneDrive/Documentos/SITEC/Cotizaciones (caratula)/Capa1Extract/2023/Febrero/CTQ-844 BUSSMANN (SERVICIO DE SUMINISTRO DE RUEDA).csv')
cot3 = pd.read_csv('C:/Users/52473/OneDrive/Documentos/SITEC/Cotizaciones (caratula)/Capa1Extract/2023/Julio/CTQ-952 ZF (EMBALAJE LINEA EP5).csv')
cot4 = pd.read_csv('C:/Users/52473/OneDrive/Documentos/SITEC/Cotizaciones (caratula)/Capa1Extract/2023/Septiembre/CTQ-995 PIZZA HUT (SERVICIO DE FABRICACIÓN DE PUERTA PARA ÁREA DEL SITE).csv')
cot5 = pd.read_csv('C:/Users/52473/OneDrive/Documentos/SITEC/Cotizaciones (caratula)/Capa1Extract/2023/Noviembre/CTQ-1042 VALEO (SUMINISTRO DE VIDRIO TEMPLADO).csv')

Los dataframes provienen de diccionarios, es decir, las columnas: range, majorDimensio, provienen de la funcion de extracción de Google Sheets.

La columna importantes es unicamente "value", trabajaremos con esa información.



# Limpieza

Eliminaremos valores nulos y la sección de términos y condiciones.

In [2]:
#Creamos funciones de transformación


#Funcion para eliminar valores nulos en este contexto:

def dropnull(df):
    j=0
    while j<len(df):
        if '[]' in df['values'][j]:
            #print(f"el registro {j} es nulo")
            df=df.drop(index = df.index[j])
            df.reset_index(drop = True, inplace = True)
            #print(f"dimension interaccion {n}")
            #print(f"dimension real: {len(cotizacion1)}")
        else:
            #print(f"registro {j} no nulo")
            #print(f"dimension real: {len(cotizacion1)}")
            j+=1
    return df

def dropextrainf(df):
    n=0
    while n< len(df):
        if re.search("T[EÉ]RMINOS",df['values'][n]) != None:    #Añadimos funciones regex
            #print(f"A partir del registro {n} tiene los terminos y condiciones")
            df = df.drop(index = df.index[n:])
            break
        else:
            #print(f"La información de este registro {n} es importante")
            n+=1
    return df

In [67]:
#funcion general de limpieza
#print(dropnull(dropextrainf(cot2)))     #No importa el orden de la composición de funciones.

def limpieza1(df): return dropnull(dropextrainf(df))

# TRANSFORMACIÓN 



Los datos extraidos los trabajaremos como CSV´s. La estructura de la columna values de los dataframes, son cadenas de caracteres que representen toda una fila del documento en formato CSV.

In [68]:
#Creamos un dataframe donde las columnas serán las filas del archivo y las filas los elementos separados por comas.
 
#Los valores faltantes para el tamaño del dataframe lo rellenamos con valores nulos
def filetodataframe(cleanfile):
    separaciones = [len(re.split(',',cleanfile['values'][filas])) for filas in range(len(cleanfile))]   
    df = pd.DataFrame(np.zeros((max(separaciones),len(cleanfile))))
    #print(separaciones)
    #print(f" el dataframe EES: {df}")
    for j in range(len(cleanfile)):    
        #print(f"Seguimos con la fila {j}")
        fila = re.split(',',cleanfile['values'][j])
        fila.extend(np.full((len(df)-len(fila)),''))
        df[j] = fila
        #print(f"el tamano de la lista a incorporar es: {len(fila)} y la longitud del dataframe es: {len(df)}, tienen que empatar")
    return df

#Al agregar los ceros, convertimos un dataframe que tiene registros tipo str y tipo entero, hay que tener cuidado
#cambiamos la funcion np.zeros por np.full((ejex,ejey,ejez), valor_en_cada_registro_de_toda_la_estructura)


# Limpieza de registros


Los registros presentan corchetes, comillas, dobles comillas y espacios que vamos a requerir de quitar y dejar unicamente las palabras importantes para que el motor de busqueda detecte coincidencias y extraiga lo necesario

In [70]:
#Limpieza de registros: 
def cleanregister(dataframe):
    carex = r'[\[\'\"\]\:]'
    #s = registro.replace(' ','')
    #print(f"el registro: {registro} , registro limpio: {re.sub(carex,'',s)}")
    df = pd.DataFrame(np.full((len(dataframe),len(dataframe.columns)),''))
    for columnas in range(len(dataframe.columns)):
        for filas in range(len(dataframe)):
            #print(f"fila: {a} , columna: {b}")
            df.iloc[filas,columnas] = re.sub(carex,'',dataframe.iloc[filas,columnas].strip())
            #dfclean.iloc[a,b] = df.iloc[a,b]  
    return df


# Extracción

extraemos los campos importantes y creamos un tablas a partir de la extracción.

Los conceptos a extrar serán:

PORYECTO:
FOLIO:
UBICACIÓN:
CONTACTO:

clave, descripción/partida , unidad, cantidad, P.U, cantidad

COSTO TOTAL:


Los campos en mayusculas son conceptos simples, es decir, tiene un unico dato/valor.

Los campos que estan en minusculas consecutivamente son valores que por simplicidad podemos hacer una tabla propiamente, ya que en el documento excel estan presentados en tablas y su orden hará mas fácil la extracción.



La tabla de cotizaciones tendra la siguiente forma:

folio, fecha_cotizacion, proyecto, ubicación, contacto, estado, costo_total.

Crearemos otra tabla llamada proyectos, ligada a la tabla cotizaciones:

folio_cotización, n_partidas, cantidad/valor_por_partida, costo_total, descripción.

In [71]:
#BUSQUEDA
def busqueda(dataframe):
    metdata_cot = ['FOLIO','PROYECTO', 'CONTACTO' ,'UBICACI[OÓ]N', 'FECHA' , 'UNE' , 'COSTO TOTAL' ]  #palabras a buscar
    coordenadas = np.empty((len(metdata_cot),5),dtype= object)              
    for palabra in range(len(metdata_cot)):
        for columnas in range(len(dataframe.columns)):
            for fila in range(len(dataframe) ):
                if re.search(metdata_cot[palabra], dataframe.iloc[fila,columnas]) != None:
                    #guardamos la palabra y su coordenada de término de la palabra
                    coordenadas[palabra] = [metdata_cot[palabra],
                                            fila,
                                            columnas,
                                            re.search(metdata_cot[palabra],dataframe.iloc[fila,columnas]).start(),
                                            re.search(metdata_cot[palabra],dataframe.iloc[fila,columnas]).end()]
                    dataf = pd.DataFrame(coordenadas, columns= ['palabra', 'fila_df' , 'columna_df' , 'c_inicio' , 'c_fin' ])
    
    return dataf
     
            
  #cambiamos la definicion de coordenadas de: np.full((len(metdata_cot),5),'') a lista = np.empty((3, 5), dtype=object) , ya que al definir la lista como la 
  #primer opcion, al querer asignar cadenas a estos elementos, como se definió que sus valores fueran vacios en np.full ('') asume que la longitud de los caracteres
  # de los elementoe es de unidad 1, por eso cuando se quiere añadir una cadena de caracteres, solo toma la primera letra

In [98]:
#EXTRACCIÓN:

#El campo costo total y el valor van por separados, dentro de la misma columna, pero a diferencia de los otros
#campos, este no va junto con su valor, por lo que tenemos que tratarlo aparte


def ext(dataframe, cordinates):
#definimos el paquete (esquema) para ordenar
    f = np.empty((len(cordinates),1),dtype = object)  
    for fila in range(len(cordinates)):
            if fila < 6: 
                f[fila] = [dataframe.iloc[cordinates['fila_df'][fila],cordinates['columna_df'][fila]][cordinates['c_fin'][fila]:]]    
                #print(f)
    else:
            for n in range(6,len(dataframe),1):
                if re.search('\$',dataframe.iloc[n,len(dataframe.columns)-1]) != None: 
                    #print(dfclean.iloc[n,33])
                    #print(dfclean.iloc[n,33][2:] + dfclean.iloc[n+1,33][0:]) 
                    f[fila] = [dataframe.iloc[n,len(dataframe.columns)-1][2:] + dataframe.iloc[n+1,len(dataframe.columns)-1][0:]] 
    return list(f)




#Extracción de lote de 5:

Las funciones creadas son:

1.- limpieza1(dataframe)

2.-filetodata(dataframe)

3.-cleanregister(dataframe)

4.- busqueda(dataframe) output: coordinates

5.- extraccion(dataframe, coordinates)

In [100]:
#Realizamos el proceso de extracción para estas 5 cotizaciones

def extract(dataframe):
    a = limpieza1(dataframe)
    #print(f"proceso eliminación valores vacios y términos y condiciones.")
    b = filetodataframe(a)
    #print("proceso de transformación a dataframe realizado con éxito.")
    c = cleanregister(b)
    #print("proceso de limpieza de registros realizado con éxito.")
    d = busqueda(c)
    #print("proceso de busqueda de patrones realizada con éxito")
    e = ext(c,d)
    #print("proceso de extracción realizado con éxito")
    return e
    
    

In [102]:
#Definimos la tabla de los metadatos:
# folio, proyecto, contacto, ubicacion, fecha, une, costo total

cotizaciones = pd.DataFrame(columns= ['folio','proyecto','contacto','ubicacion','fecha','une','costo total'])


cots = [cot1,cot2,cot3,cot4,cot5]

#cots = {nombre_cotizacion : cotizacion for nombre_cotizacion,cotizacion in zip(,)}


for cot in cots:
    f = extract(cot)
    #print(f"transformado: {f}")
    cotizaciones = cotizaciones.append(pd.Series(f, index=cotizaciones.columns), ignore_index=True)


cotizaciones

C:\Users\52473\AppData\Local\Temp\ipykernel_9028\270240923.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cotizaciones = cotizaciones.append(pd.Series(f, index=cotizaciones.columns), ignore_index=True)
C:\Users\52473\AppData\Local\Temp\ipykernel_9028\270240923.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cotizaciones = cotizaciones.append(pd.Series(f, index=cotizaciones.columns), ignore_index=True)
C:\Users\52473\AppData\Local\Temp\ipykernel_9028\270240923.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cotizaciones = cotizaciones.append(pd.Series(f, index=cotizaciones.columns), ignore_index=True)
C:\Users\52473\AppData\Local\Temp\ipykernel_9028\270240923.py:15: FutureWarning: The frame.append method is d

,folio,proyecto,contacto,ubicacion,fecha,une,costo total
0,[ CTQ-000787],[ SERVICIO DE DESCARGA Y ARRASTRE DE MAQUINA],[ BENITO ZAMORA / ELECTROSERVICIOS ],[ PLANTA TOPRE],[ 03/01/2023],[ QUERETARO.],[ 74467.000 ]
1,[ CTQ-000844],[ SERVICIO DE SUMINISTRO DE RUEDA],[ MIGUEL SALCEDO],[ BUSSMANN],[ 21/02/023],[ QUERETARO],[ 3380.00 ]
2,[ CTQ-000952],[ EMBALAJE LINEA EP5],[ ANA GUTIERREZ],[ PLANTA ZF DIRECCIONES ],[ 11/07/2023],[ QUERETARO],[ 481000.000 ]
3,[ CTQ-000995],[ SERVICIO DE FABRICACIÓN DE PUERTA PARA ÁRE...,[ PAULO SOLIS],[ PIZZA HUT],[ 20/09/2023],[ QUERETARO],[ 33000.00 ]
4,[ CTQ-001042],[ SUMINISTRO DE VIDRIO TEMPLADO],[ ANTONIO GOMEZ],[ VALEO KAPEC],[ 09/11/2023],[ QUERÉTARO.],[ 18000.00 ]


In [93]:
#AREA PRUEBA


